<a href="https://www.kaggle.com/code/colewelkins/give-me-words?scriptVersionId=129226480" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
 [code]

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries and set paths

In [ ]:
import torch
import glob
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from skimage.io import imsave
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import nn, optim
from torch.optim.lr_scheduler import OneCycleLR
import torch.nn.functional as F
import matplotlib.patches as patches
import gc

# Constants
PREFIX = '/kaggle/input/vesuvius-challenge-ink-detection/train/1/'
BUFFER = 30
Z_START = 27
Z_DIM = 10
TRAINING_STEPS = 30000
LEARNING_RATE = 0.001
BATCH_SIZE = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a Dataset class for the 3d x-ray scan
class XRayDataset(Dataset):
    def __init__(self, file_paths, transform=None):
        self.file_paths = file_paths
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.file_paths[idx]), dtype=np.float32)/65535.0
        if self.transform:
            image = self.transform(image)
        return image

# Create a DataLoader for the 3d x-ray scan
file_paths = sorted(glob.glob(PREFIX+"surface_volume/*.tif"))[Z_START:Z_START+Z_DIM]
dataset = XRayDataset(file_paths, transform=torch.from_numpy)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Display a few images
images = [dataset[i] for i in range(5)]
fig, axes = plt.subplots(1, len(images), figsize=(15, 3))
for image, ax in zip(images, axes):
    ax.imshow(image, cmap='gray')
    ax.set_xticks([]); ax.set_yticks([])
fig.tight_layout()
plt.show()

# Load the label data
label = torch.from_numpy(np.array(Image.open(PREFIX+"inklabels.png"))).gt(0).float().to(DEVICE)

# Now we'll create a dataset of subvolumes. We use a small rectangle around the letter "P" for our evaluation, and we'll exclude those pixels from the training set. (It's actually a Greek letter "rho", which looks similar to our "P".)
rect = (1100, 3500, 700, 950)
fig, ax = plt.subplots()
ax.imshow(label.cpu())
patch = patches.Rectangle((rect[0], rect[1]), rect[2], rect[3], linewidth=2, edgecolor='r', facecolor='none')
ax.add_patch(patch)
plt.show()

# Define SubvolumeDataset class

In [ ]:
# Define SubvolumeDataset class
class SubvolumeDataset(Dataset):
    def __init__(self, image_stack, label, ink_pixels):
        self.image_stack = image_stack
        self.label = label
        self.ink_pixels = ink_pixels

    def __len__(self):
        return len(self.ink_pixels)

    def __getitem__(self, idx):
        y, x = self.ink_pixels[idx]
        z_start = max(0, Z_START - BUFFER)
        z_end = min(self.image_stack.shape[0], Z_START + BUFFER)
        subvolume = self.image_stack[z_start:z_end, y-BUFFER:y+BUFFER, x-BUFFER:x+BUFFER]
        inklabel = self.label[y, x]
        return subvolume, inklabel
    
    # Create a DataLoader for the subvolumes
ink_pixels = torch.stack((label.nonzero(as_tuple=True)[0][BUFFER:-BUFFER], label.nonzero(as_tuple=True)[1][BUFFER:-BUFFER]), dim=1)
ink_pixels = ink_pixels[~((ink_pixels[:,0] > rect[1]) & (ink_pixels[:,0] < rect[1]+rect[3]) & (ink_pixels[:,1] > rect[0]) & (ink_pixels[:,1] < rect[0]+rect[2]))]
subvolume_dataset = SubvolumeDataset(torch.stack([dataset[i] for i in range(len(dataset))]).to(DEVICE), label, ink_pixels)
subvolume_dataloader = DataLoader(subvolume_dataset, batch_size=BATCH_SIZE, shuffle=True)

def process_file(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip().split(',')

def find_common_elements(file1, file2):
    file1_data = set(process_file(file1))
    file2_data = set(process_file(file2))

    common_elements = file1_data & file2_data

    return common_elements

# Define InkDetectionModel class

In [ ]:
class InkDetectionModel(nn.Module):
    def __init__(self):
        super(InkDetectionModel, self).__init__()
        self.conv1 = nn.Conv3d(1, 64, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv3d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv3d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.fc = nn.Linear(256 * 4 * 4 * 4, 1)  # This needs to be adjusted based on the output size of the conv layers and input image size

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.sigmoid(self.fc(x))
        return x

# Load Data

In [ ]:
mask = np.array(Image.open(PREFIX+"mask.png").convert('1'))
label = torch.from_numpy(np.array(Image.open(PREFIX+"inklabels.png"))).gt(0).float().to(DEVICE)
images = [np.array(Image.open(filename), dtype=np.float32)/65535
    for filename in sorted(glob.glob(PREFIX+"surface_volume/*.tif"))]
image_stack = torch.from_numpy(np.stack(images)).float().to(DEVICE)

# Compute ink pixels

In [ ]:
# Compute ink pixels
ink_pixels = np.argwhere(mask)
np.random.shuffle(ink_pixels)

# Prepare Dataset and Dataloader
dataset = SubvolumeDataset(image_stack, label, ink_pixels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Prepare Dataset and Dataloader

In [ ]:
dataset = SubvolumeDataset(image_stack, label, ink_pixels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Initialize Model, Optimizer, and Scheduler

In [ ]:
model = InkDetectionModel().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = OneCycleLR(optimizer, max_lr=LEARNING_RATE, steps_per_epoch=len(dataloader), epochs=TRAINING_STEPS//len(dataloader))

# Training Loop

In [ ]:
# Training Loop
model.train()
loss_values = []  # to store loss values
best_loss = np.inf  # initialize best loss to infinity
early_stopping_counter = 0  # counter for early stopping
EARLY_STOPPING_STEPS = 10  # number of steps with no improvement after which training will be stopped
for step in tqdm(range(TRAINING_STEPS)):
    subvolumes, inklabels = next(iter(dataloader))
    subvolumes, inklabels = subvolumes.to(DEVICE), inklabels.to(DEVICE)

    optimizer.zero_grad()
    outputs = model(subvolumes)
    loss = nn.BCELoss()(outputs, inklabels)
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    torch.cuda.empty_cache()

    loss_values.append(loss.item())

    # print loss every 1000 steps
    if step % 1000 == 0:
        print(f"Step {step}, Loss: {loss.item()}")

# Save Model

In [ ]:
if loss.item() < best_loss:
        best_loss = loss.item()
        torch.save(model.state_dict(), 'best_model.pth')
        early_stopping_counter = 0  # reset counter
else:
    early_stopping_counter += 1
        
   # early stopping
if early_stopping_counter >= EARLY_STOPPING_STEPS:
        print("Early stopping...")
        break

    # free up memory
    del subvolumes, inklabels, outputs
    torch.cuda.empty_cache()
    gc.collect()

# Perform inference and generate a probability map

In [ ]:
# Evaluate Model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for subvolumes, inklabels in dataloader:
        subvolumes, inklabels = subvolumes.to(DEVICE), inklabels.to(DEVICE)
        outputs = model(subvolumes)
        predicted = (outputs > 0.5).float()
        total += inklabels.size(0)
        correct += (predicted == inklabels).sum().item()

    print(f'Accuracy of the network on the test subvolumes: {100 * correct / total}%')

# Evaluation of Model on the Rectangle Region

In [ ]:
eval_dataset = SubvolumeDataset(image_stack, label, pixels_inside_rect)
eval_loader = data.DataLoader(eval_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Apply the model to the evaluation dataset

In [ ]:
model.eval()
outputs = []
with torch.no_grad():
    for subvolumes, _ in tqdm(eval_loader):
        subvolumes = subvolumes.to(DEVICE)
        output = model(subvolumes)
        outputs.append(output.cpu())

# Concatenate the outputs into a single tensor

In [ ]:
outputs = torch.cat(outputs)

# Create an empty array to hold the predicted ink locations

In [ ]:
predicted_ink = np.zeros(inside_rect.shape, dtype=bool)

# For each pixel in the rectangle, if the model's output is greater than 0.5, mark it as ink

In [ ]:
for pixel, output in zip(pixels_inside_rect, outputs):
    y, x = pixel
    predicted_ink[y, x] = output.item() > 0.5

# Display the predicted ink locations

In [ ]:
plt.imshow(predicted_ink, cmap='gray')
plt.title('Predicted Ink Locations')
plt.show()

# Thresholding and Visualization

In [ ]:
# Thresholding and Visualization
THRESHOLD = 0.4  # Threshold for classifying a subvolume as ink

# Evaluation of Model on the Rectangle Region
eval_dataset = SubvolumeDataset(image_stack, label, ink_pixels)
eval_loader = DataLoader(eval_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Apply the model to the evaluation dataset
model.eval()
outputs = []
with torch.no_grad():
    for subvolumes, _ in tqdm(eval_loader):
        subvolumes = subvolumes.to(DEVICE)
        output = model(subvolumes)
        outputs.append(output.cpu())

# Concatenate the outputs into a single tensor
outputs = torch.cat(outputs)

# Create an empty array to hold the predicted ink locations
predicted_ink = np.zeros(mask.shape, dtype=bool)

# For each pixel in the rectangle, if the model's output is greater than THRESHOLD, mark it as ink
for pixel, output in zip(ink_pixels, outputs):
    z, y, x = pixel
    predicted_ink[z, y, x] = output.item() > THRESHOLD

# Display the predicted ink locations
plt.imshow(predicted_ink[0], cmap='gray')  # Change the index based on which slice you want to visualize
plt.title('Predicted Ink Locations')
plt.show()

# Run-Length Encoding (RLE) for Submission

In [ ]:
# Run-Length Encoding (RLE) for Submission
def rle(output):
    pixels = np.where(output.flatten() > THRESHOLD, 1, 0).astype(np.uint8)
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return ' '.join(str(x) for x in runs)

rle_output = rle(predicted_ink)
print("Id,Predicted\na," + rle_output + "\nb," + rle_output, file=open('submission.csv', 'w'))